# K-최근접 이웃 다중 분류

## 교제 데이터 사용

In [21]:
import pandas as pd
import numpy as np

# pd.read_csv() : 교제 csv데이터를 읽는 함수
fish = pd.read_csv('https://bit.ly/fish_csv_data')
# 상위 5개 데이터 확인
fish.head()

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340


In [22]:
fish_input = fish[['Weight','Length','Diagonal','Height','Width']].to_numpy()

In [23]:
fish_target = fish['Species'].to_numpy()

In [24]:
# 중복을 제거하여 생선 종류를 확인
print(pd.unique(fish['Species']))

['Bream' 'Roach' 'Whitefish' 'Parkki' 'Perch' 'Pike' 'Smelt']


## 샘플링 편향 문제 해결

In [25]:
# scikit-learn 라이브러리에서 제공하는 함수로 데이터를 훈련 세트와 테스트 세트로 분할하기 위해 사용
from sklearn.model_selection import train_test_split

# random_state 값을 주는 이유는 항상 동일한 실행 결과 값을 얻기 위해서 사용함
# random_state 매개변수는 데이터를 무작위로 섞을 때 사용되는 시드(seed)값을 지정
train_input, test_input, train_target, test_target = train_test_split(
    fish_input, fish_target, random_state=42)

In [26]:
print(train_input.shape, test_input.shape)

(119, 5) (40, 5)


In [27]:
print(train_target.shape, test_target.shape)

(119,) (40,)


In [28]:
print(test_target)

['Perch' 'Smelt' 'Pike' 'Whitefish' 'Perch' 'Bream' 'Smelt' 'Roach'
 'Perch' 'Pike' 'Bream' 'Whitefish' 'Bream' 'Parkki' 'Bream' 'Bream'
 'Perch' 'Perch' 'Perch' 'Bream' 'Smelt' 'Bream' 'Bream' 'Bream' 'Bream'
 'Perch' 'Perch' 'Whitefish' 'Smelt' 'Smelt' 'Pike' 'Perch' 'Perch' 'Pike'
 'Bream' 'Perch' 'Roach' 'Roach' 'Parkki' 'Perch']


## 데이터 전처리

In [29]:
# np.mean() : numpy에서 제공하는 평균을 구하는 함수
# np.std() : numpy에서 제공하는 표준 편차를 구하는 함수
# axis = 0 : 각 열(axis=0)에 대해 계산을 지정하는 매개변수
mean = np.mean(train_input, axis=0)
std = np.std(train_input, axis=0)

In [30]:
print(mean, std)

[393.27226891  28.49663866  31.2697479    8.86248403   4.44828655] [355.27097574  10.67118899  11.5129708    4.04941829   1.64047206]


In [31]:
# 데이터 표준화
train_scaled = (train_input - mean) / std
test_scaled = (test_input - mean) / std

## K-최근접 이웃 다중 분류 수행하기

In [46]:
# KNeighborsClassifier : scikit-learn 라이브러리에서 제공되는 함수, K 최근접 이웃 분류기는 주어진 데이터의 이웃들을 기반으로 분류를 수행하는 머신러닝 알고리즘입니다.
from sklearn.neighbors import KNeighborsClassifier

#  K-최근접 이웃 분류 객체 생성
kn = KNeighborsClassifier(n_neighbors=3)
# train_data로 훈련
kn.fit(train_scaled, train_target)

KNeighborsClassifier(n_neighbors=3)

In [47]:
# 훈련된 모델로 test 데이터 분류
kn.predict(test_scaled)

array(['Perch', 'Smelt', 'Pike', 'Perch', 'Perch', 'Bream', 'Smelt',
       'Roach', 'Perch', 'Pike', 'Bream', 'Perch', 'Bream', 'Parkki',
       'Bream', 'Bream', 'Perch', 'Perch', 'Roach', 'Bream', 'Smelt',
       'Bream', 'Bream', 'Bream', 'Bream', 'Perch', 'Perch', 'Perch',
       'Smelt', 'Smelt', 'Pike', 'Perch', 'Roach', 'Pike', 'Bream',
       'Perch', 'Roach', 'Perch', 'Parkki', 'Perch'], dtype=object)

In [48]:
# 모델의 정확도를 출력
kn.score(train_scaled, train_target)

0.8907563025210085

In [49]:
# 모델의 정확도를 출력
kn.score(test_scaled, test_target)

0.85

In [50]:
# kn.predict_proba : 각 클래스에 속할 확률을 예측하는 데 사용, 예측된 확률은 모델이 예측한 각 클래스에 속할 확률을 포함하는 배열로 반환
proba = kn.predict_proba(test_scaled[:5])
print(np.round(proba, decimals=3))

[[0.    0.    1.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    1.    0.   ]
 [0.    0.    0.    1.    0.    0.    0.   ]
 [0.    0.    0.667 0.    0.333 0.    0.   ]
 [0.    0.    0.667 0.    0.333 0.    0.   ]]
